In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn import linear_model
import math

In [18]:
def travel_prob(distances, populations):
    """ 
    Finds transition probabilities for traveling from each county to each other county.
    
    Input:
        distances: matrix with distances between each pair of county
        populations: array with population of each county (indices correspond to those of distances matrix)
    
    Output:
        pi: matrix of transition probabilities
    """
    l = len(populations)
    pi = np.zeros((l, l))
    for i in range(l - 1):
        pop_i = populations[i]
        for j in range(i + 1, l):
            pop_j = populations[j]
            dist = distances[i][j]
            pi_for = (pop_j / pop_i) / dist
            pi_back = (pop_i / pop_j) / dist
#            threshold = pop_i / pop_j
#             if (threshold <= 0.5):
#                 pi_for = (1 / threshold) / dist
#                 pi_back = pi_for
#             elif (threshold >= 2):
#                 pi_for = threshold / dist
#                 pi_back = pi_for
#             else:
#                 pi_for = pop_i / dist
#                 pi_back = pop_j / dist
            pi[i, j] = pi_for
            pi[j, i] = pi_back
    return pi

Finding Constants
* need constants for recovery rate, disease death rate, sickness rate
* recovery rate and disease death rate should be time-dependent
* sickness rate would just depend on susceptible, infected, and total population (until a vacine shows up)

In [2]:
# For now, anything having to do with population arrays hasn't been finalized because we need to decide the exact mechanics
# of how we're gonna pass in the data. We need a function that'll return the total population right now, and an array that 
# gives us the population for every day that we have data for. Once we starting doing actual runs from a starting point,
# we can probably say N = S + I + R.

class County:
    def __init__(self, covid_data, population_data, location_data, starting_data, alpha):
        """
        starting_data format: [S, I, R, D]
        """
        self.covid_data = covid_data
        self.population_data = population_data
        self.location_data = location_data
        # self.mu = natural death rate
        # self.lambda = natural birth rate
        self.alpha = alpha
        self.S = starting_data[0]
        self.I = starting_data[1]
        self.R = starting_data[2]
        self.D = starting_data[3]
        self.data = starting_data
        self.day = 0
        self.lat = location_data[0]
        self.long = location_data[1]
        
    def set_alpha(self, val):
        self.alpha = val
    
    def beta(self):
        return self.beta
    
    def gamma(self):
        return self.gamma
    
    def delta(self):
        return self.delta
    
    def lambd(self):
        return self.lambd
    
    def mu(self):
        return self.mu
    
    def prev_S(self):
        return self.prev_S
    
    def prev_I(self):
        return self.prev_I
    
    def prev_R(self):
        return self.prev_R
    
    def prev_D(self):
        return self.prev_D
    
    def prev_N(self):
        return self.prev_N # need to return population at current time after we figure out specifics of population data
    
    def S(self):
        return self.S
    
    def I(self):
        return self.I
    
    def R(self):
        return self.R
    
    def D(self):
        return self.D
    
    def N(self):
        return self.N # need to return population at current time after we figure out specifics of population data
    
    def set_sd(self, std):
        """
        Takes in standard deviation array in the following format: [beta, gamma, delta, lambd, mu]
        """
        self.beta_sd = std[0] ** 0.5
        self.gamma_sd = std[1] ** 0.5
        self.delta_sd = std[2] ** 0.5
        self.lambd_sd = std[3] ** 0.5
    
    def least_squares(self):
        """"
        Uses linear least squares to find estimates for parameters delta, gamma, and beta.
        """
        cases = []
        deaths = []
        recovered = []
        for date in self.covid_data:
            cases.append(self.covid_data[data][0])
            deaths.append(self.covid_data[data][1])
            recovered.append(self.covid_data[data][2])
        cases = np.array(cases)
        deaths = np.array(deaths)
        recovered = np.array(recovered)
        # susceptible array
        # population array
        
        # Find delta using D(t + 1) - D(t)
        deaths_diff = np.diff(death)
        cases_adjusted = cases[:-1].copy()
        self.delta = np.linalg.inv(cases_adjusted @ cases_adjusted.T) @ cases_adjusted @ deaths_diff.T
        
        # Find gamma using R(t + 1) - (1 - mu)R(t)
        recovered_mu = (1 - self.mu) * recovered[:-1].copy()
        recovered_next = recovered[1:].copy()
        recovered_diff = recovered_next - recovered_mu
        self.gamma = np.linalg.inv(cases_adjusted @ cases_adjusted.T) @ cases_adjusted @ recovered_diff.T
        
        # Find beta using S(t + 1) - (lambda + (1 - mu)S(t))
        beta_A = -1 * np.divide(np.multiply(susceptible[:-1].copy(), cases[:-1].copy()), population[:-1].copy()) # edit
        susceptible_mu = (1 - self.mu) * susceptible[:-1].copy()
        pop_lambd = self.lambd * population[:-1].copy() # edit
        susceptible_next = susceptible[:1].copy()
        susceptible_diff = susceptible_next - pop_lambd - susceptible_mu
        self.beta = np.linalg.inv(beta_A @ beta_A.T) @ beta_A @ susceptible_diff.T
        
    def step(self):
        self.day += 1
        self.prev_S = self.S
        self.prev_I = self.I
        self.prev_R = self.R
        self.prev_D = self.D
        self.prev_N = self.N
        beta = np.random.normal(self.beta, self.beta_sd)
        gamma = np.random.normal(self.gamma, self.gamma_sd)
        delta = np.random.normal(self.delta, self.delta_sd)
        mu = np.random.normal(self.mu, self.mu_sd)
        lambd = np.random.normal(self.lambd, self.lambd_sd)
        self.S = lambd * population - beta * prev_S * prev_I / prev_N + (1 - mu) * prev_S # edit
        self.I = beta * prev_S * prev_I / prev_N + (1 - gamma - delta - mu) * prev_I
        self.R = gamma * prev_I + (1 - self.mu) * prev_R
        self.D = delta * prev_I + prev_D
        self.N = self.S + self.I + self.R # TENTATIVE
    
    def run(self, num):
        for i in range(num):
            step()

#         lm = linear_model.LinearRegression()
#         _ = lm.fit(cases, recovered)
#         gamma = lm.coef_
#         gamma_int = lm.intercept_
#         lm = linear_model.LinearRegression()
#         _ = lm.fit(cases, deaths)
#         delta = lm.coef_
#         delta_int = lm.intercept_

In [3]:
class Country:
    def __init__(self, covid_data, pop_data, dist_data):
        self.covid_data = covid_data
        self.pop_data = pop_data
        self.dist_data = dist_data
        self.counties = []
        beta = []
        gamma = []
        delta = []
        lambd = []
        mu = []
        parameters = np.array([beta, gamma, delta, lambd, mu])
        for state in covid_data:
            for county in covid_data[state]:
                new_county = County(covid_data[state][county], pop_data[state][county], dist_data[state][county], start, alpha) # edit
                self.counties.append(new_county)
                new_county.least_squares()
                beta.append(new_county.beta())
                gamma.append(new_county.gamma())
                delta.append(new_county.gamma())
                lambd.append(new_county.lambd())
                mu.append(new_county.mu())
        st_devs = np.std(parameters, axis = 1)
        self.counties = np.array(self.counties)
        County.set_sd(self.counties, st_devs) # not sure if this is the correct usage
            
    def distance(self, county1, county2, alpha1, alpha2):
        """
        Calculates the distance between two given counties.
        """
        
        r = 6373
        
        lat1 = math.radians(county1.lat)
        long1 = math.radians(county1.long)
        lat2 = math.radians(county2.lat)
        long2 = math.radians(county2.long)

        dlong = lon2 - lon1
        dlat = lat2 - lat1

        a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlong / 2)**2

        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return r * c
    
    def travel_val(self, county1, county2):
        """
        Calculates the amount of travel from county1 to county2 and from county2 to county1.
        """
        d = distance(county1, county2)
        forward = np.random.poisson(alpha1 * county2.prev_N() / d)
        backward = np.random.poisson(alpha2 * county1.prev_N() / d)
        return forward, backward
    
    def step(self):
        County.run(self.counties) # not sure if this is the correct usage
        for i in range(len(self.counties) - 1):
            county1 = self.counties[i]
            for j in range(i + 1, len(self.counties)):
                county2 = self.counties[j]
                forward, backward = travel_val(county1, county2)
                county1.S += backward - forward
                county2.S += forward - backward
                # insert R changes
        
    def run(self, num):
        for i in range(num):
            step()

Things to still figure out:
* population array: specifics of the data structure and how to access it - need to fix the County class accordingly
* alpha: heuristic for giving each county its own alpha value to find diffusion values (depends on quarantining methods, suburb vs. big city, etc.)
* diffusion for S vs. R: total population to use for each one (either S and R or N)